In [3]:
# import necessary libraries  (you can import just before using as well)

import win32com.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Get the database object
notesSession = win32com.client.Dispatch(r'Notes.NOTESSESSION')
titanicDb = notesSession.GetDatabase("Dev-01", "developers\ewerfel\Collabsphere\collabsphere1.nsf")
print(type(notesSession))
titanicDb.title

In [ ]:
view = titanicDb.GetView("Passengers")
view.Name

In [ ]:
# Get the itemList from the first document in view
doc = view.GetFirstDocument
itemList = []
for i in doc.Items:
    itemName = i.name
    if itemName[0] != "$" and itemName != "Form":
        itemList.append(itemName)
itemList

In [ ]:
# Populate dataframe with data from every document in view 

count = 0   
titanicList = []
doc = view.GetFirstDocument
while not doc == None: 
    passengerDictionary = {}
    for i in itemList:        
        if (i in ["PassengerId", "Pclass", "Survived"]):
            passengerDictionary[i] = int(doc.GetItemValue(i)[0])
        else:
            passengerDictionary[i] = doc.GetItemValue(i)[0]
    titanicList.append(
        passengerDictionary
    )       
    count = count + 1
    if count % 25 == 0:
        #break
        print(count)
    doc = view.GetNextDocument(doc)
titanicDf = pd.DataFrame(titanicList)

titanicDf.head(30)

In [ ]:
titanicDf.tail(10)

In [ ]:
titanicDf.shape

In [ ]:
titanicDf.describe()

In [ ]:
titanicDf.dtypes

In [ ]:
titanicDf[titanicDf["Age"]==""]


In [ ]:
titanicDf

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
titanicDf

In [ ]:
women_survival = titanicDf[titanicDf.Sex == 'female']["Survived"]  # number of women who survived
rate_women = sum(women_survival)/len(women_survival)    # number of women who survived / all women


men_survival = titanicDf[titanicDf.Sex == 'male']["Survived"]  # number of men who survived
rate_men = sum(men_survival)/len(men_survival)    # of women who surived / all women

print("Women: " + str(rate_women))
print("Men: " + str(rate_men))

In [ ]:
classes = titanicDf.groupby(['Pclass']).agg({'Survived':['sum','count','mean']})


In [ ]:
titanicDf.groupby(['Embarked']).agg({'Survived':['sum','count','mean']})

In [ ]:
titanicAgeDf = titanicDf[titanicDf["Age"]!=""]
titanicAgeDf

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
titanicAgeDf.describe()

In [ ]:
didNotSurvive = titanicAgeDf['Age'][titanicDf['Survived']==0]
survived = titanicAgeDf['Age'][titanicDf['Survived']==1]
plt.hist([didNotSurvive,survived], label=['Survived: No','Survived: Yes'])
plt.legend(loc='upper right')
plt.title('Relative Frequency of Age');


In [ ]:
lookup = {"C":"Cherbourg","S":"Southampton","Q":"Queenstown"}
titanicDf["FullEmbarked"]=titanicDf["Embarked"].replace(lookup)
titanicDf.head()

In [ ]:
myVar=titanicDf[titanicDf["PassengerId"]==2]["FullEmbarked"].iloc[0]
myVar

In [ ]:
doc = view.GetFirstDocument
while not doc == None: 
    passengerId = doc.GetItemValue("PassengerId")
    fullEmbarked = titanicDf[titanicDf["PassengerId"]==passengerId]["FullEmbarked"].iloc[0]
    doc.ReplaceItemValue("FullEmbarked",[fullEmbarked])
    doc.Save(True, True)
    doc = view.GetNextDocument(doc)

In [ ]:
count=0
doc = view.GetFirstDocument
while not doc == None: 
    passengerId = doc.GetItemValue("PassengerId")
    fullEmbarkedSeries = titanicDf.loc[titanicDf["PassengerId"]==passengerId,"FullEmbarked"]
    print(doc.getItemValue("Name")[0])
    if( fullEmbarkedSeries.size > 0):
        fullEmbarked = fullEmbarkedSeries.iloc[0]
        print(fullEmbarked)
        doc.ReplaceItemValue("FullEmbarked",fullEmbarked)
        doc.Save(True, True)
    else:
        print("Not found")
    count = count + 1
    if count%50 == 0:
        #break
        print(count)
    doc = view.GetNextDocument(doc)

In [ ]:
destFile = r'C:\Users\ewerfel\Documents\Collabsphere\titanic.xlsx'
titanicDf.to_excel(destFile)